<a href="https://colab.research.google.com/github/chikilivighneshshastry/colab_files/blob/main/jobdata_api_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from datetime import datetime, timedelta

published_since = (datetime.now() - timedelta(days=2)).strftime('%Y-%m-%d')
print(published_since)
cookies = {
    'csrftoken': '5LSZIfHHmDnSIaZKDsN0hGYXaWQhOJGV',
}

headers = {
    'accept': 'application/json',
    'accept-language': 'en-US,en;q=0.9',
    'priority': 'u=1, i',
    'referer': 'https://jobdataapi.com/api/schema/swagger-ui/',
    'sec-ch-ua': '"Google Chrome";v="137", "Chromium";v="137", "Not/A)Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36',
    # 'cookie': 'csrftoken=5LSZIfHHmDnSIaZKDsN0hGYXaWQhOJGV',
}
parameters = {
    'country_code' : 'IN',
    # 'page' : '',
    # 'page_size' : 50,
    'published_since' : published_since
}

response = requests.get('https://jobdataapi.com/api/jobs/', cookies=cookies, headers=headers,params=parameters)

print(response)

In [ ]:
import json
# print(json.dumps(response.json()))
data = response.json()
json.dumps(response.json())

In [ ]:
results = data['results']
print(results)
len(results)

In [ ]:
import json
job_post = results[0]

json.dumps(job_post['cities'])

In [ ]:
import pandas as pd

df = pd.DataFrame(results)

df


In [ ]:
# @title experience_level vs salary_min

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['salary_min'].value_counts()
    for x_label, grp in df.groupby('experience_level')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('experience_level')
_ = plt.ylabel('salary_min')

In [ ]:
latest_published = df['published'].max()
earliest_published = df['published'].min()

print(f"Latest published date: {latest_published}")
print(f"Earliest published date: {earliest_published}")

# 3. Calculate the difference directly
# latest_published and earliest_published are already datetime objects
time_difference = latest_published - earliest_published

In [ ]:
df_city_fil = df[df['has_remote'].apply(lambda x: x)]
print(df_city_fil)

In [ ]:
df.head(2)

In [ ]:
company_name = df['company'][0]
print(company_name)

**Reasoning**:
Import the sqlite3 library, define the database file name, establish a connection, and create a cursor.



In [ ]:
import sqlite3

db = './jobs_data.db'
conn = sqlite3.connect(db)
cursor = conn.cursor()

## Create table if not exists

### Subtask:
Define and create the table in the database with appropriate columns if it doesn't already exist.


**Reasoning**:
Define and create the table in the database with appropriate columns if it doesn't already exist.



In [ ]:
create_table_sql = """
CREATE TABLE IF NOT EXISTS raw_jobdataapi_data (
    id INTEGER PRIMARY KEY,
    ext_id TEXT,
    company TEXT,
    title TEXT,
    location TEXT,
    types TEXT,
    cities TEXT,
    states TEXT,
    countries TEXT,
    regions TEXT,
    has_remote BOOLEAN,
    published datetime,
    description TEXT,
    experience_level TEXT,
    application_url TEXT,
    language TEXT,
    salary_min INTEGER,
    salary_max INTEGER,
    salary_currency TEXT,

    verified BOOLEAN default False,
    page_content_html TEXT,
    is_expried BOOLEAN,
    created_at datetime default CURRENT_TIMESTAMP,
    updated_at datetime default CURRENT_TIMESTAMP
)
"""
cursor.execute(create_table_sql)
conn.commit()

**Reasoning**:
Iterate through the DataFrame, check for existing records by id, and insert new records into the SQLite database, converting complex types to JSON strings.



In [ ]:
import json
import pandas as pd
import sqlite3
from datetime import datetime, timedelta

published_since = (datetime.now() - timedelta(days=2)).strftime('%Y-%m-%d')
# published_since = datetime.now() - timedelta(days=2)
db = './jobs_data.db'
conn = sqlite3.connect(db)

# df = df[df['published']<20min] # correct this code

# Get existing IDs from the database
existing_ids = pd.read_sql(f"SELECT id FROM raw_jobdataapi_data where published > {published_since}", conn)['id'].tolist() # get last 20 min id's

# Filter out rows that already exist in the database
df_to_insert = df[~df['id'].isin(existing_ids)].copy()

# Convert complex data types to JSON strings in the DataFrame
for col in ['company', 'types', 'cities', 'states', 'countries', 'regions']:
    df_to_insert[col] = df_to_insert[col].apply(lambda x: json.dumps(x) if isinstance(x, (dict, list)) else None)

# Insert the filtered DataFrame into the SQLite database
df_to_insert.to_sql('raw_jobdataapi_data', conn, if_exists='append', index=False)

conn.commit()

**Reasoning**:
Close the connection to the SQLite database.



In [ ]:
conn.close()

In [ ]:
!pip install playwright
!playwright install

# playright code

In [ ]:
# prompt: get all jobs with verified = false read from db and verifiy job posts going to url using playright and update db with page_content_html and is_expried status based upon title in content (expried or not) and change verifiy to true

# !pip install playwright
# !playwright install

import sqlite3
from playwright.sync_api import sync_playwright
import re

db = './jobs_data.db'
conn = sqlite3.connect(db)
cursor = conn.cursor()

# Get all jobs with verified = false
cursor.execute("SELECT id, application_url FROM raw_jobdataapi_data WHERE verified = FALSE")
jobs_to_verify = cursor.fetchall()

p = sync_playwright().start()
browser = p.chromium.launch(headless=True)
page = browser.new_page()

for job_id, job_url in jobs_to_verify:
    try:
        page.goto(job_url, timeout=60000)  # Set a timeout for page loading
        page_content = page.content()

        # Check for "expired" in the title or content
        is_expired = bool(re.search(r'(expired|closed|no longer available)', page_content, re.IGNORECASE))

        # Update the database
        update_sql = """
        UPDATE raw_jobdataapi_data
        SET verified = TRUE,
            page_content_html = ?,
            is_expried = ?,
            updated_at = CURRENT_TIMESTAMP
        WHERE id = ?
        """
        cursor.execute(update_sql, (page_content, is_expired, job_id))
        conn.commit()
        print(f"Verified job {job_id} at {job_url}. Expired status: {is_expired}")

    except Exception as e:
        print(f"Error verifying job {job_id} at {job_url}: {e}")
        # Optionally, you might want to log this error or mark the job differently in the DB

browser.close()

conn.close()

In [ ]:
import sqlite3
import asyncio # Import asyncio
from playwright.async_api import async_playwright # Import async_playwright
import re

# Database configuration
db = './jobs_data.db'

# Define the async function to run the Playwright operations
async def verify_job_posts():
    conn = sqlite3.connect(db)
    cursor = conn.cursor()

    # Get all jobs with verified = FALSE
    # Ensure your table has a 'verified' column (e.g., INTEGER, 0 for FALSE, 1 for TRUE)
    # And 'page_content_html' (TEXT), 'is_expired' (INTEGER), 'updated_at' (DATETIME)
    cursor.execute("SELECT id, application_url FROM raw_jobdataapi_data WHERE verified = 0")
    jobs_to_verify = cursor.fetchall()

    if not jobs_to_verify:
        print("No unverified jobs found to process.")
        conn.close()
        return

    # Use async_playwright for async operations
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        for job_id, job_url in jobs_to_verify:
            try:
                print(f"Attempting to verify job {job_id} at {job_url}...")
                await page.goto(job_url, timeout=60000)  # Set a timeout for page loading
                page_content = await page.content()

                # Check for "expired" in the title or content
                # Added a check for common "not found" or "404" page indicators
                is_expired = bool(re.search(r'(expired|closed|no longer available|404 Not Found|Page Not Found)', page_content, re.IGNORECASE))
                # Also check page status for explicit 404/500 errors
                if page.status != 200:
                    is_expired = True

                # Update the database
                update_sql = """
                UPDATE raw_jobdataapi_data
                SET verified = ?, -- Use 1 for TRUE
                    page_content_html = ?,
                    is_expried = ?, -- Use 1 for TRUE, 0 for FALSE
                    updated_at = CURRENT_TIMESTAMP
                WHERE id = ?
                """
                # SQLite stores booleans as 0 (False) or 1 (True)
                cursor.execute(update_sql, (1, page_content, 1 if is_expired else 0, job_id))
                conn.commit()
                print(f"Verified job {job_id}. Expired status: {is_expired}")

            except Exception as e:
                print(f"Error verifying job {job_id} at {job_url}: {e}")
                # You might want to update the 'verified' status to indicate a failure
                # e.g., verified = -1 (for error) or add an 'error_message' column
                # For now, we just print the error and continue
                conn.rollback() # Rollback any partial updates for this job if an error occurred

        await browser.close()
        print("Browser closed.")

    conn.close()
    print("Database connection closed.")

# Run the async function
if __name__ == "__main__":
    # Create a dummy table for demonstration if it doesn't exist

    # This is how you run an async function in a synchronous context
    await verify_job_posts()

# Update predefined tabels

__________

## All locations in a table
* save countries , states and citys in db along with codes like country code and similar

## All skills in skills table
- get skills details from `https://www.kaggle.com/datasets/arbazkhan971/allskillandnonskill/data?select=skills.csv` and update db

# Company
- get compnay details from Jobdata_api or any other source and update db

# Job category
- update db with all predefined job category

> ## how to handle if any of the above data we get from api is missmatching or not avalibale with us  
>> mostly we will have issue with `# Location and company details`  handle that.  
> like by add new details in db or identiflying similar name with any method

> need to use techniques like.  
1 Fuzzy matching.  
2 third-party services to match or online free data set.   
3 ML model

# Other tables :
  ### Job Type

- Full-time

- Contract

- Part-time

- Internship

### Work Model

- Onsite

- Remote

- Hybrid

### Experience Level

- Intern/New Grad
notes

- Entry Level
notes

- Mid Level
notes

- Senior Level
notes

- Lead/Staff
notes

- Director/Executive




In [ ]:
#first go to url with request lib or any async lib and update db and make 2nd flow with playwright as below
#get all jobs with verified = false and verifiy job posts going to url using playwright and update db with page_content_html and is_expried status based upon title in content (expried or not) and change verifiy to true

# read data from this raw_table (last 2 days data)
# read main_jobs_data table (last 2 days data)
# compare both by id and url and remove commen
# for unique run data_manupulation worker --> based upon city name get
# get city and (location details) , work_type,clean published (date) and check if date is behyod the current data, from json
# description -->html to text , company details from json to compnay table mapping
# send data to ai to tag a category,get skills,yeas_of_exp (min and max),musthave qulification,prefred qulifications,,

#save this data in main jobs table and also inject data in main jobs dependcy tables like category and jobpost matching

In [ ]:
# when user request data
# get all jobpost from main jobs table after filtering category (selected by user)
# check if this jobposts already in user_score table if not run scoring worker and append data to table
# send data to front end
# delete expried jobpost fromt his table when delete_expried_worker run

In [ ]:
import json
import pandas as pd
import sqlite3

db = './jobs_data.db'
conn = sqlite3.connect(db)
cursor = conn.cursor()
create_table_sql = """
CREATE TABLE IF NOT EXISTS job_posts (
    id TEXT PRIMARY KEY,
    jobtitle TEXT,
    jobdataapi_id TEXT,
    title TEXT,
    location TEXT,
    jobdescription TEXT,
    company_id TEXT,
    published_date TEXT, -- Storing ISO 8601 strings, e.g., 'YYYY-MM-DD HH:MM:SS'
    published_time_ux INTEGER, -- Unix timestamp (seconds since epoch)
    experience_level TEXT,
    salary_min INTEGER,
    salary_max INTEGER,
    salary_currency TEXT,
    job_language TEXT,
    application_url TEXT,
    country_id TEXT,
    -- Define Foreign Key constraint (optional if company_details table isn't created yet)
    FOREIGN KEY (company_id) REFERENCES company_details(id)
        ON DELETE NO ACTION ON UPDATE NO ACTION
);
"""
# Execute the SQL command
cursor.execute(create_table_sql)
conn.commit()
print(f"Table 'job_posts' created successfully in {db}")

In [ ]:
main_table_ids = pd.read_sql(f"SELECT jobdataapi_id,application_url FROM job_posts where published > {published_since}", conn)['jobdataapi_id'].tolist()
raw_table_ids = pd.read_sql(f"SELECT id FROM raw_jobdataapi_data where published > {published_since}", conn)['id'].tolist()
# Filter out rows that already exist in the database
df_to_insert = df[~df['id'].isin(existing_ids)].copy()

# Classify Job category using job title and JD similarity score with our category data in table


      This is very Importent if need accuracy then use better embading model or fine-tune embadding model for accuracy

-------------------------------------



### Need to extact from JD
------------

- Responsibilities  
- Qualification -
              1 Required
             2 preferred
- Benifits
----------
- Years of Experiance -- `Min_y and Max_y`
- skills -- > can be extracted using our skills table
- if remort in page mark it as remote
- Salary

In [ ]:
!pip install httpx aiohttp requests

In [ ]:
import asyncio
import aiohttp
from bs4 import BeautifulSoup
import pandas as pd

# --- Function to Clean HTML into Plain Text ---
def clean_html_text(html_content: str) -> str:
    """
    Cleans HTML content by removing tags and normalizing whitespace.
    """
    if not html_content:
        return ""
    try:
        soup = BeautifulSoup(html_content, 'html.parser')

        # Get all the text
        text = soup.get_text()

        # Break into lines and remove leading/trailing space on each
        lines = (line.strip() for line in text.splitlines())

        # Break multi-word lines into phrases and join them with a single space
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))

        # Join everything back together with a single space and remove any final excess whitespace
        cleaned_text = ' '.join(chunk for chunk in chunks if chunk)

        return cleaned_text
    except Exception as e:
        print(f"Error cleaning text: {e}")
        return "Could not clean HTML content."


# --- Asynchronous Function to Fetch a URL ---
async def fetch_url(session: aiohttp.ClientSession, url: str):
    """
    Asynchronously fetches a single URL and returns the URL and its text content.
    Includes error handling for network or HTTP issues.
    """
    try:
        # The 'async with' statement ensures the connection is closed properly.
        async with session.get(url, timeout=10) as response:
            # raise_for_status() will raise an exception for 4xx/5xx responses
            response.raise_for_status()
            print(f"SUCCESS: Fetched {url} with status {response.status}")
            html_content = await response.text()
            return url, html_content
    except aiohttp.ClientError as e:
        # Handles client-side errors (e.g., connection error, timeout)
        print(f"ERROR: Could not fetch {url}. Reason: {e}")
        return url, None
    except Exception as e:
        # Handles other potential errors
        print(f"UNEXPECTED ERROR for {url}. Reason: {e}")
        return url, None


# --- Main Orchestrator Function ---
async def main():
    """
    Main function to run the fetching and processing concurrently.
    """
    # List of 10 URLs to fetch.
    # I've included a delayed URL to demonstrate the "as they arrive" behavior.
    urls = [
        "http://python.org",
        "https://www.google.com",
        "https://github.com",
        "https://www.wikipedia.org",
        "https://httpbin.org/delay/3",  # This one will intentionally finish later
        "https://aiohttp.readthedocs.io",
        "https://www.python-requests.org",
        "https://stackoverflow.com",
        "https://httpbin.org/status/404", # This one will fail
        "https://developer.mozilla.org",
    ]

    # Create a DataFrame with url, url_id, and url_active
    df = pd.DataFrame(urls, columns=['url'])
    df['url_id'] = df.index
    df['url_active'] = False
    df['page_content'] = ''

    # aiohttp recommends a single session for all requests
    async with aiohttp.ClientSession() as session:
        # Create a list of tasks (coroutines) to run
        tasks = [fetch_url(session, url) for url in df['url']]

        # --- The Magic: asyncio.as_completed() ---
        # This function returns an iterator that yields futures as they complete.
        # This is exactly what you want: process results as they arrive.
        print("--- Starting to fetch URLs. Output will appear as it arrives ---")

        for future in asyncio.as_completed(tasks):
            # We 'await' the future to get its result (url, html_content)
            url, html_content = await future

            if html_content:
                # If we got content, clean it and update the DataFrame
                cleaned_text = clean_html_text(html_content)

                # Find the row for the URL and update it
                df.loc[df['url'] == url, ['url_active', 'page_content']] = [True, cleaned_text]

                print("-" * 50)
                print(f"✅ RESULT for: {url}")
                print(f"Cleaned Content (first 200 chars): {cleaned_text[:200]}...")
                print("-" * 50, "\n")
            else:
                # If fetching failed, html_content will be None
                # url_active remains False
                print("-" * 50)
                print(f"❌ FAILED for: {url}")
                print("-" * 50, "\n")

    print("\n--- Final DataFrame ---")
    print(df)


if __name__ == "__main__":
    # This is the standard way to run an asyncio program
    asyncio.run(main())

In [ ]:
import asyncio
import time
from concurrent.futures import ThreadPoolExecutor
import httpx
import aiohttp
from typing import List, Dict, Any

class AsyncThreadPoolBenchmark:
    def __init__(self, urls: List[str], num_requests: int = 100):
        self.urls = urls
        self.num_requests = num_requests

    # AIOHTTP - Pure Async (no threads)
    async def aiohttp_async_test(self) -> Dict[str, Any]:
        errors = 0

        connector = aiohttp.TCPConnector(limit=100, limit_per_host=50)
        timeout = aiohttp.ClientTimeout(total=30)

        async with aiohttp.ClientSession(connector=connector, timeout=timeout) as session:
            start_time = time.time()

            async def fetch_url(session, url):
                try:
                    async with session.get(url) as response:
                        content = await response.read()
                        return response.status, len(content)
                except Exception as e:
                    return None, str(e)

            tasks = []
            for i in range(self.num_requests):
                url = self.urls[i % len(self.urls)]
                tasks.append(fetch_url(session, url))

            results = await asyncio.gather(*tasks, return_exceptions=True)
            end_time = time.time()

            for result in results:
                if isinstance(result, Exception) or (hasattr(result, '__len__') and len(result) == 2 and result[0] is None):
                    errors += 1

            total_time = end_time - start_time

        return {
            'total_time': total_time,
            'requests_per_second': self.num_requests / total_time,
            'avg_time_per_request': total_time / self.num_requests,
            'errors': errors,
            'successful_requests': self.num_requests - errors
        }





    async def run_benchmark(self, thread_workers: List[int] = [4]) -> Dict[str, Dict[str, Any]]:
        print(f"Running benchmark with {self.num_requests} requests to {len(self.urls)} URLs")
        print("=" * 80)

        results = {}

        # Test AIOHTTP Pure Async
        print("Testing AIOHTTP (Pure Async - no threads)...")
        results['aiohttp_async'] = await self.aiohttp_async_test()


        # # Test with different thread pool sizes
        # for workers in thread_workers:
        #     print(f"Testing AIOHTTP with ThreadPool ({workers} workers)...")
        #     results[f'aiohttp_threads_{workers}'] = await self.aiohttp_threadpool_test(workers)

        #     print(f"Testing HTTPX with ThreadPool ({workers} workers)...")
        #     results[f'httpx_threads_{workers}'] = await self.httpx_threadpool_test(workers)

        return results

    def print_results(self, results: Dict[str, Dict[str, Any]]):
        print("\n" + "=" * 90)
        print("ASYNC vs THREADPOOL BENCHMARK RESULTS")
        print("=" * 90)

        # Sort by requests per second (descending)
        sorted_results = sorted(results.items(),
                              key=lambda x: x[1]['requests_per_second'],
                              reverse=True)

        print(f"{'Method':<25} {'Workers':<8} {'Total Time':<12} {'Req/Sec':<12} {'Avg/Req':<12} {'Errors':<8}")
        print("-" * 90)

        for method, data in sorted_results:
            workers = data.get('max_workers', 'N/A')
            print(f"{method:<25} "
                  f"{workers:<8} "
                  f"{data['total_time']:<12.2f} "
                  f"{data['requests_per_second']:<12.2f} "
                  f"{data['avg_time_per_request']:<12.4f} "
                  f"{data['errors']:<8}")

        print("\n" + "=" * 90)
        winner = sorted_results[0]
        print(f"WINNER: {winner[0].upper()}")
        print(f"Best performance: {winner[1]['requests_per_second']:.2f} requests/second")

        # Analysis
        print("\n" + "=" * 90)
        print("ANALYSIS:")

        # Compare async vs threaded for each library
        aiohttp_async = results.get('aiohttp_async', {}).get('requests_per_second', 0)
        # httpx_async = results.get('httpx_async', {}).get('requests_per_second', 0)

        # # Find best threaded performance for each
        # aiohttp_threaded_best = max([v['requests_per_second'] for k, v in results.items() if k.startswith('aiohttp_threads_')], default=0)
        # httpx_threaded_best = max([v['requests_per_second'] for k, v in results.items() if k.startswith('httpx_threads_')], default=0)

        # print(f"AIOHTTP: Pure Async ({aiohttp_async:.2f} req/s) vs Best Threaded ({aiohttp_threaded_best:.2f} req/s)")
        # if aiohttp_threaded_best > aiohttp_async:
        #     improvement = (aiohttp_threaded_best / aiohttp_async - 1) * 100
        #     print(f"  → Threading is {improvement:.1f}% FASTER for AIOHTTP")
        # else:
        #     improvement = (aiohttp_async / aiohttp_threaded_best - 1) * 100
        #     print(f"  → Pure Async is {improvement:.1f}% FASTER for AIOHTTP")

        # print(f"HTTPX: Pure Async ({httpx_async:.2f} req/s) vs Best Threaded ({httpx_threaded_best:.2f} req/s)")
        # if httpx_threaded_best > httpx_async:
        #     improvement = (httpx_threaded_best / httpx_async - 1) * 100
        #     print(f"  → Threading is {improvement:.1f}% FASTER for HTTPX")
        # else:
        #     improvement = (httpx_async / httpx_threaded_best - 1) * 100
        #     print(f"  → Pure Async is {improvement:.1f}% FASTER for HTTPX")

        print("\nKEY INSIGHTS:")
        print("- Pure async is more memory efficient (no thread overhead)")
        print("- ThreadPool can be faster when async libraries have connection limits")
        print("- Threading adds overhead but can bypass some async bottlenecks")
        print("- Optimal thread count depends on network latency and server response times")
        print("=" * 90)

async def main():
    # Test URLs - mix of different response times
    test_urls = [
        "https://httpbin.org/json",
        "https://jsonplaceholder.typicode.com/posts/1",
        "https://api.github.com/users/octocat",
        "https://httpbin.org/delay/0.1",
        "https://jsonplaceholder.typicode.com/users/1",
        "https://httpbin.org/uuid",
    ]

    # Create benchmark instance
    benchmark = AsyncThreadPoolBenchmark(test_urls, num_requests=1000)

    # Run the benchmark with different thread counts
    results = await benchmark.run_benchmark(thread_workers=[1])

    # Print results
    benchmark.print_results(results)

if __name__ == "__main__":
    # Install required packages:
    # pip install httpx aiohttp requests

    print("Starting Async vs ThreadPool Benchmark...")
    print("This will test pure async vs threading approaches for HTTP requests")
    print()

    await main()